In [1]:
import pandas as pd
import openpyxl
import shutil

In [2]:
# Read data from the source Excel file
source_file = 'EdREhmann_S25ProductFilterwCost.xlsx'
columns = ['Name','Material','Cost','Size','Dimension','UPC','New/Carryover']
data = pd.read_excel(source_file,header=1,usecols=columns) # 0-indexed

# Drop rows that are Carryover
carryover = data[data['New/Carryover'] == 'Carryover'].index
data.drop(carryover, inplace=True)

#Replace NaN with empty strings
data['Dimension'] = data['Dimension'].fillna('')
data['Size'] = data['Size'].fillna('')
data['Material'] = data['Material'].fillna('')

# Calculate Price unit based on Cost (pu = (cost*2)*.85)
data['Price Unit'] = (data['Cost']*2)*.85

In [3]:
# Create Marc approved name for use on customer receipt
data['Name'] = data[['Material','Size','Dimension']].aggregate('-'.join, axis=1)

# Remove the now unneeded columns
data.drop(columns=['Material','Size','Dimension','New/Carryover'],inplace=True)

# Rename to match Clover format
data.rename(columns={'UPC':'Product Code'},inplace=True)

# Add columns to match Clover format, some with default values
data.insert(0,'Clover ID','')
data.insert(2,'Alternate Name','')
data.insert(3,'Price','0')
data.insert(4,'Price Type','FIXED')
data.insert(6,'Tax Rates','DEFAULT')
data.insert(7,'SKU','')
data.insert(8,'Modifier Groups','')
data.insert(9,'Quantity','0')
data.insert(10,'Printer Labels','')
data.insert(11,'Hidden','FALSE')
data.insert(12,'Non-revenue item','FALSE')


In [4]:
# Copy the template file to the output file. We'll overwrite the 'Items' sheet
template = 'CloverInventoryTemplate_Small.xlsx'
output_file = 'CloverInventoryoutput.xlsx'
shutil.copy2(template,output_file)

'CloverInventoryoutput.xlsx'

In [5]:
# Write the data to the output file
with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer,sheet_name='Items', index=False)